In [24]:
import win32com.client
instCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
print(instCpCybos.IsConnect)

1


In [25]:
import win32com.client
instCpStockCode = win32com.client.Dispatch("CpUtil.CpStockCode")
print(instCpStockCode.GetData(1, 0))

동화약품


In [26]:
# import win32com.client

# # Create object
# instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")

# # SetInputValue
# instStockChart.SetInputValue(0, "A012450") #한화에어로스페이스
# instStockChart.SetInputValue(1, ord('1'))
# instStockChart.SetInputValue(2, 20230627)
# instStockChart.SetInputValue(3, 20210627)
# instStockChart.SetInputValue(4, 2000)
# instStockChart.SetInputValue(5, (0, 1, 2, 3, 4, 5, 6,7,8))
# instStockChart.SetInputValue(6, ord('d')) #분 당
# instStockChart.SetInputValue(9, ord('1')) #1 분

# # BlockRequest
# instStockChart.BlockRequest()

# # GetHeaderValue
# numData = instStockChart.GetHeaderValue(3)
# numField = instStockChart.GetHeaderValue(1)

# # GetDataValue
# for i in range(numData):
#     for j in range(numField):
#         print(instStockChart.GetDataValue(j, i), end=" ")
#     print("")

In [27]:
# #종목정보 가져오기(최초)

# import win32com.client
# import pandas as pd 

# rows = list()

# CPE_MARKET_KIND = {'KOSPI':1,  'KOSDAQ':2}
# instCpCodeMgr = win32com.client.Dispatch("CpUtil.CpCodeMgr")

# for key, value in CPE_MARKET_KIND.items():
#     codeList = instCpCodeMgr.GetStockListByMarket(value)
#     for code in codeList:
#         name = instCpCodeMgr.CodeToName(code)
#         sectionKind = instCpCodeMgr.GetStockSectionKind(code)
#         row = [code, name, key, sectionKind]
        
#         rows.append(row)

# print('모든 종목을 불러왔습니다')

# stockitems = pd.DataFrame(data= rows, columns=['code','name', 'section','sectionKind'])
# stockitems.loc[stockitems['sectionKind'] == 10, 'section'] =   'ETF'
# stockitems.to_csv('stockitems.csv', index=False)

# print('파일을 저장하였습니다.')

In [30]:
##주가정보 가져오기  (최초 생성)

import win32com.client
import pandas as pd 
import time
import datetime

column_dailychart = ['code', 'section', 'date', 'open', 'high', 'low', 'close',
                     'vol', 'value', 'n_stock', 'agg_price', 'foreign_rate','agency_buy', 'agency_netbuy']

stockitems = pd.read_csv('stockitems.csv')

instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
nCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")

row = list(range(len(column_dailychart)))
rows = list()

# instStockChart.SetInputValue(0, "A012450")
instStockChart.SetInputValue(1, ord('1'))
# instStockChart.SetInputValue(2, (datetime.datetime.today() - datetime.timedelta(days=1)).strftime("%Y%m%d"))
instStockChart.SetInputValue(2, '20210628')
instStockChart.SetInputValue(3, '20210628')
instStockChart.SetInputValue(5, (0, 1, 2, 3, 4, 5, 8, 9, 12, 13, 17, 20, 21))
instStockChart.SetInputValue(6, ord('m'))
instStockChart.SetInputValue(9, ord('1'))
    
for idx, stockitem in stockitems.iterrows():

    remain_request_count =  nCpCybos.GetLimitRemainCount(1)
    print(stockitem['code'], stockitem['name'], '남은 요청 : ', remain_request_count)
    
    if remain_request_count == 0:
        print('남은 요청이 모두 소진되었습니다. 잠시 대기합니다.')

        while True:
            time.sleep(2)
            remain_request_count =  nCpCybos.GetLimitRemainCount(1)
            if  remain_request_count > 0:
                print('작업을 재개합니다. (남은 요청 : {0})'.format(remain_request_count))
                break
            print('대기 중...')
            
    instStockChart.SetInputValue(0, stockitem['code'])

    # BlockRequest
    instStockChart.BlockRequest()

    # GetHeaderValue
    numData = instStockChart.GetHeaderValue(3)
    numField = instStockChart.GetHeaderValue(1)

    # GetDataValue
    for i in range(numData):
        row[0] =  stockitem['code']
        row[1] = stockitem['section']  # 코스피, 코스닥, ETF 여부
        row[2] = instStockChart.GetDataValue(0, i)  # 날짜
        row[3] = instStockChart.GetDataValue(1, i)  # 시가
        row[4] = instStockChart.GetDataValue(2, i)  # 고가
        row[5] = instStockChart.GetDataValue(3, i)  # 저가
        row[6] = instStockChart.GetDataValue(4, i)  # 종가
        row[7] = instStockChart.GetDataValue(5, i)  # 거래량
        row[8] = instStockChart.GetDataValue(6, i)  # 거래대금
        row[9] = instStockChart.GetDataValue(7, i)  # 상장주식수
        row[10] = instStockChart.GetDataValue(8, i)  # 시가총액
        row[11] = instStockChart.GetDataValue(9, i)  # 외국인 보율비율
        row[12] = instStockChart.GetDataValue(10, i)  # 기관순매수
        row[13] = instStockChart.GetDataValue(11, i)  # 기관누적순매수
        rows.append(list(row))  
        
print('데이터를 모두 불러왔습니다.')

dailychart= pd.DataFrame(data = rows, columns= column_dailychart)
dailychart[['value' ,'agg_price']] = (dailychart[['value' ,'agg_price']]/1000000).astype(int)
dailychart = dailychart.sort_values(by=['code','date'])
dailychart.to_csv('dailychart.csv', index=False)

print('모든 데이터를 저장하였습니다.')



# dailychart = pd.DataFrame(data = rows, columns= column_dailychart)

# dailychart[['value' ,'agg_price']] /= 1000000
# dailychart[['value' ,'agg_price']] = dailychart[['value' ,'agg_price']].astype(int)
# dailychart = dailychart.append(dailychart_prev)
# dailychart = dailychart.sort_values(by=['code','date'])

# dailychart.to_csv('dailychart.csv', index=False)
# print('데이터를 모두 불러왔습니다.')

A012450 한화에어로스페이스 남은 요청 :  60
데이터를 모두 불러왔습니다.
모든 데이터를 저장하였습니다.
